In [1]:
import duckdb
import json
from datetime import date, timedelta
import pandas as pd

In [2]:
with open("credentials.json") as f:
    d = json.load(f)

pg = d["postgres"]
aws = d["aws"]
bucket = "s3://carlos-s3-bdet-ducklake" 

def secreto(con):
    print("Iniciando")
    #con = duckdb.connect( )
    con.sql("INSTALL ducklake; LOAD ducklake;")
    con.sql("INSTALL spatial; LOAD spatial;")
    con.sql("INSTALL httpfs; LOAD httpfs;")
    con.sql("INSTALL postgres; LOAD postgres;")

    print("Librerias cargadas")

    con.sql(f"""
        CREATE OR REPLACE SECRET secreto_s3 (
        TYPE s3,
        KEY_ID '{aws["login"]}',
        SECRET '{aws["password"]}',
        REGION 'eu-central-1'
    )
    """)

    print("Secreto s3 creado")

    con.sql(f"""
        CREATE OR REPLACE SECRET secreto_postgres (
        TYPE postgres,
        HOST '{pg["host"]}',
        PORT {pg["port"]},
        DATABASE '{pg["schema"]}',
        USER '{pg["login"]}',
        PASSWORD '{pg["password"]}'
        )
    """)

    print("Secreto postgres creado")

    con.sql("""
        CREATE OR REPLACE SECRET secreto_ducklake (
            TYPE ducklake,
            METADATA_PATH '',
            METADATA_PARAMETERS MAP {'TYPE': 'postgres', 'SECRET': 'secreto_postgres'}
        );
        """)
    
    print("secreto ducklake creado")

    con.sql(f"""
        ATTACH 'ducklake:secreto_ducklake' AS mobility_ducklake (DATA_PATH '{bucket}', OVERRIDE_DATA_PATH TRUE) """)
    con.sql("""
        USE mobility_ducklake """)
    
    print("fin")

In [12]:
con = duckdb.connect()
secreto(con)

Iniciando
Librerias cargadas
Secreto s3 creado
Secreto postgres creado
secreto ducklake creado
fin


In [15]:
date_start = '2023-05-08'
date_end = '2023-05-08'

con.sql(f"SELECT * FROM intermediate_daily_features WHERE date BETWEEN '{date_start}' AND '{date_end}' ORDER BY date").df()

,date,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,2023-05-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.0,0.0


In [8]:
polygon = 'POLYGON((40.9 -1.7, 40.9 0.9, 37.8 0.9, 37.8 -1.7, 40.9 -1.7))'
date_start = '2023-05-08'
date_end = '2023-05-08'

query = f"""--sql
                CREATE OR REPLACE TABLE silver_trips_filtrado AS
                WITH zona_estudio AS (
                    SELECT distrito_id
                    FROM silver_geometries
                    WHERE ST_Within(
                        centroide, 
                        ST_Transform(
                            ST_GeomFromText('{polygon}'),
                            'EPSG:4326', 
                            'EPSG:25830'
                        )
                    )
                )
                SELECT 
                    t.date,
                    t.period,
                    t.origin_id,
                    t.destination_id,
                    t.travels
                FROM silver_trips t
                INNER JOIN zona_estudio origen ON t.origin_id = origen.distrito_id
                INNER JOIN zona_estudio destino ON t.destination_id = destino.distrito_id
                WHERE 
                    t.date BETWEEN '{date_start}' AND '{date_end}';"""

con.sql(query)

In [14]:
con.sql("SELECT * FROM silver_trips_filtrado LIMIT 10")

┌──────┬────────┬───────────┬────────────────┬─────────┐
│ date │ period │ origin_id │ destination_id │ travels │
│ date │ int32  │  varchar  │    varchar     │ double  │
├──────┴────────┴───────────┴────────────────┴─────────┤
│                        0 rows                        │
└──────────────────────────────────────────────────────┘

In [5]:
poligono = 'POLYGON((40.9 -1.7, 40.9 0.9, 37.8 0.9, 37.8 -1.7, 40.9 -1.7))'

con.sql(f"""--sql
        SELECT 
            distrito_id
        FROM silver_geometries
        WHERE ST_Within(
            centroide, 
            ST_Transform(
                    ST_GeomFromText('{poligono}'),
                'EPSG:4326', 
                'EPSG:25830'
            )
        );""")

┌─────────────┐
│ distrito_id │
│   varchar   │
├─────────────┤
│ 0200501     │
│ 0200701     │
│ 0202301     │
│ 0202902     │
│ 0203706     │
│ 0205601     │
│ 0207403     │
│ 0207404     │
│ 0207902     │
│ 0208301     │
│    ·        │
│    ·        │
│    ·        │
│ 4622701     │
│ 4623504     │
│ 4623601     │
│ 4623901     │
│ 4624801     │
│ 4624902     │
│ 4625003     │
│ 4626201     │
│ 4690401     │
│ NA          │
├─────────────┤
│  1092 rows  │
│ (20 shown)  │
└─────────────┘

In [23]:
con.sql(f"""--sql
        SELECT 
            distrito_id,
            centroide
        FROM silver_geometries
        WHERE ST_Within(
            centroide, 
            ST_GeomFromText('POLYGON((500000.0 4760000.0, 500000.0 4764000.0, 502400.0 4764000, 502400.0 4760000, 500000.0 4760000.0))')
        );""")

┌─────────────┬──────────────────────────────────────────────┐
│ distrito_id │                  centroide                   │
│   varchar   │                   geometry                   │
├─────────────┼──────────────────────────────────────────────┤
│ 0100201     │ POINT (502306.66342558316 4763642.870954381) │
└─────────────┴──────────────────────────────────────────────┘

In [28]:
con.sql("SELECT distrito_id, centroide FROM silver_geometries WHERE distrito_id = '2418801' LIMIT 10")

┌─────────────┬──────────────────────────────────────────────┐
│ distrito_id │                  centroide                   │
│   varchar   │                   geometry                   │
├─────────────┼──────────────────────────────────────────────┤
│ 2418801     │ POINT (293794.36424201383 4684252.104875557) │
└─────────────┴──────────────────────────────────────────────┘

In [29]:
con.sql("SELECT * FROM bronze_distritos_nombres WHERE ID = '24188' LIMIT 10")

┌─────────┬──────────────────────┐
│   ID    │         name         │
│ varchar │       varchar        │
├─────────┼──────────────────────┤
│ 24188   │ Valencia de Don Juan │
└─────────┴──────────────────────┘